### **Laboratorio 6**

- Juan Pablo Solis
- Isabella Miralles

In [15]:
import json
import re
import string
import pandas as pd
import nltk

# Descargar stopwords en español (solo la primera vez)
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = set(stopwords.words("spanish"))

def limpiar_texto(texto):
    texto = texto.lower()
    
    texto = re.sub(r"[@#']", "", texto)
    
    texto = re.sub(r"http\S+|www.\S+", "", texto)
    
    texto = re.sub(r"[^\w\sáéíóúüñ]", "", texto)
    
    texto = texto.translate(str.maketrans("", "", string.punctuation))
    
    texto = re.sub(r"\d+", "", texto)
    
    texto = " ".join([w for w in texto.split() if w not in stop_words])
    
    return texto.strip()

def cargar_tweets_archivo(archivo):
    registros = []
    buffer = ""
    nivel = 0
    
    with open(archivo, "r", encoding="utf-16") as f:
        for linea in f:
            buffer += linea
            nivel += linea.count("{")
            nivel -= linea.count("}")
            
            if nivel == 0 and buffer.strip().startswith("{"):
                try:
                    data = json.loads(buffer)
                    if "rawContent" in data:
                        registros.append({
                            "id": data.get("id"),
                            "date": data.get("date"),
                            "user": data.get("user", {}).get("username"),
                            "texto_original": data["rawContent"],
                            "texto_limpio": limpiar_texto(data["rawContent"]),
                            # 🔹 Metadatos adicionales
                            "retweets": data.get("retweetCount", 0),
                            "likes": data.get("likeCount", 0),
                            "replies": data.get("replyCount", 0),
                            "mentions": [u.get("username") for u in data.get("mentionedUsers", [])],
                            "in_reply_to": data.get("inReplyToUser", {}).get("username") if data.get("inReplyToUser") else None,
                            "conversationId": data.get("conversationId")
                        })
                except json.JSONDecodeError as e:
                    print("Error parseando:", e)
                buffer = ""
    
    return pd.DataFrame(registros)

df_trafico = cargar_tweets_archivo("data/traficogt.txt")
df_tioberny = cargar_tweets_archivo("data/tioberny.txt")

print("Tweets de traficoGT:", len(df_trafico))
print("Tweets de tioBerny:", len(df_tioberny))

df_trafico.head(), df_tioberny.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Tweets de traficoGT: 5604
Tweets de tioBerny: 5019


(                    id                       date             user  \
 0  1834236045598056867  2024-09-12 14:22:06+00:00        traficogt   
 1  1834029142565658846  2024-09-12 00:39:56+00:00     monymmorales   
 2  1834039491826180424  2024-09-12 01:21:04+00:00  animaldgalaccia   
 3  1833963729136091179  2024-09-11 20:20:01+00:00   EstacionDobleA   
 4  1833665391698092330  2024-09-11 00:34:31+00:00       CubReserva   
 
                                       texto_original  \
 0  Es comprensible la resolución... El ruso sabe ...   
 1  La corrupción de la @CC_Guatemala\nes descarad...   
 2  @PNCdeGuatemala @mingobguate @FJimenezmingob @...   
 3  @amilcarmontejo @AztecaNoticiaGT @BancadaSemil...   
 4  @soy_502 @AztecaNoticiaGT @CONAPgt @DenunciaEM...   
 
                                         texto_limpio  retweets  likes  \
 0  comprensible resolución ruso sabe engrasar maq...         0      1   
 1  corrupción ccguatemala descarada falsificación...        56     84   
 2  pn

In [16]:
df_trafico.head()

,id,date,user,texto_original,texto_limpio,retweets,likes,replies,mentions,in_reply_to,conversationId
0,1834236045598056867,2024-09-12 14:22:06+00:00,traficogt,Es comprensible la resolución... El ruso sabe ...,comprensible resolución ruso sabe engrasar maq...,0,1,0,[],None,1834236045598056867
1,1834029142565658846,2024-09-12 00:39:56+00:00,monymmorales,La corrupción de la @CC_Guatemala\nes descarad...,corrupción ccguatemala descarada falsificación...,56,84,0,[CC_Guatemala],None,1834029142565658846
2,1834039491826180424,2024-09-12 01:21:04+00:00,animaldgalaccia,@PNCdeGuatemala @mingobguate @FJimenezmingob @...,pncdeguatemala mingobguate fjimenezmingob dieg...,0,1,0,"[PNCdeGuatemala, mingobguate, FJimenezmingob, ...",PNCdeGuatemala,1834039491826180424
3,1833963729136091179,2024-09-11 20:20:01+00:00,EstacionDobleA,@amilcarmontejo @AztecaNoticiaGT @BancadaSemil...,amilcarmontejo aztecanoticiagt bancadasemilla ...,0,0,0,"[amilcarmontejo, AztecaNoticiaGT, BancadaSemil...",EstacionDobleA,1833963727575781534
4,1833665391698092330,2024-09-11 00:34:31+00:00,CubReserva,@soy_502 @AztecaNoticiaGT @CONAPgt @DenunciaEM...,aztecanoticiagt conapgt denunciaempagua munigu...,0,1,0,"[soy_502, AztecaNoticiaGT, CONAPgt, DenunciaEM...",CubReserva,1833664431084949711
